# Matrix Inversion
Matrix inversion through row manipulations.

https://docs.sympy.org/latest/tutorial/calculus.html

https://www.tutorialspoint.com/sympy/sympy_plotting.htm

https://docs.sympy.org/latest/modules/plotting.html#plotgrid-class

In [1]:
from sympy import *
import matplotlib.pyplot as plt
init_session()
init_printing(use_unicode=True)

from copy import deepcopy
from IPython.display import display, Math, Latex

# remove/comment line below to get plots in a seperate window
%matplotlib inline

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: gmpy)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



Below, we define a few routines that manipulate matrices

ShowMSimple(M1, M2, ...) simply prints the matrices M1, M2, ...
ShowM(M1, M2, ...) does the same, but prettier (using LaTeX).

RowManip(factor, startRow, targetRow, M1, M2, ...)
adds factor * startRow to target row, for all matrices M1, M2, ....
(make sure that these rows both exist, otherwise you'll get a runtime error)

RowSwap(row1, row1, M1, M2, ...)
swaps rows 1 and 2 in matrices M1, M2, ...

MultiplyRow(factor, row, M1, M2, ..) multiplies row number row with factor.

MatrixMultiplication(A, B) multiplies the matrices A and B.

Note that, if you are keen on efficiency, you'd want to replace these with some inbuilt routines, but for understanding what's going on, these hand-knitted ones are better.

In [2]:
def ShowMSimple(*matrices):
    for M in matrices:
        for row in M:
            print(row)
        print('')

def ShowM(*matrices):
    string="\(";    
    for M in matrices:
        Mstr = "   \\begin{pmatrix} "
        for row in M:
            doAmp=False;
            for a in row:
                if doAmp: Mstr += " & "
                else: doAmp = True
                Mstr += str(a)
            Mstr += " \\\\ "
        Mstr += " \\end{pmatrix} \;\;\;\;\;"
        string += Mstr
    string += "\)"
    # print(string)
    display(Latex(string))

    
# Note: we number our matrix rows and columns 1, 2, 3... not 0, 1, 2, ...
def RowManip(factor, startRow, targetRow, *matrices):
    sr=startRow-1
    tr=targetRow-1
    for M in matrices:
        for i in range(0, len(M[tr])):
            M[tr][i] = M[tr][i]+factor*M[sr][i]
            
# Note: we number our matrix rows and columns 1, 2, 3... not 0, 1, 2, ...
def RowSwap(row1, row2, *matrices):
    r1 = row1-1
    r2 = row2-1
    for M in matrices:
        temprow = M[r2]
        M[r2]=M[r1]
        M[r1]=temprow
        
def MultiplyRow(factor, row, *matrices):
    r=row-1
    for M in matrices:
        for i in range(0, len(M[r])):
            M[r][i] = factor*M[r][i]
            
def MultiplyMatrices(A, B):
    # I'm doing no checking of dimensions - make sure you pass sensible matrices
    # (or add a check yourself)
    #ShowM(A,B)
    rows = len(A)
    cols = len(B[0])
    C = [[0 for i in range(0,cols)] for j in range(0,rows)] # init results matrix
    
    for m in range(0, rows):
        for n in range(0, cols):
            for k in range(0, len(A[m])):
                C[m][n] += A[m][k] * B[k][n]
                   
    return C
            


Below we define two matrices; first the one we want to invert, second the identity matrix.

In [3]:
M33= [[1, 2, 3]
     ,[2, 3, 4]
     ,[2, 1, 1]]

I  = [[1, 0, 0]
     ,[0, 1, 0]
     ,[0, 0, 1]]

In [4]:
ShowM(M33, I)

<IPython.core.display.Latex object>

Or aim is to convert the first matrix into the identity matrix by row manipulations. By performing the same row manipulations on what is now the identity matrix, we'll find the inverse of the first matrix.

First we will transform the matrix 
$$ M = 
\begin{pmatrix}
a & b & c \\
d & e & f \\
g & h & k 
\end{pmatrix}
$$
into the shape
$$
\begin{pmatrix}
a' & b' & c' \\
0 & e' & f' \\
0 & h' & k' 
\end{pmatrix}
$$
(where the primes as in $c'$ is just generic for "not necessarilty the same number than $c$", it has nothing to do with derivatives).

then to
$$
\begin{pmatrix}
a' & 0 & c'' \\
0 & e'' & f'' \\
0 & 0 & k'' 
\end{pmatrix}
$$

then to
$$
\begin{pmatrix}
a' & 0 & 0 \\
0 & e'' & 0 \\
0 & 0 & k'' 
\end{pmatrix}
$$

then to
$$
\begin{pmatrix}
1 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 1 
\end{pmatrix}
$$
using row manipulations. <b>By performing the identical row manipulations on the unit matrix, we will obtain the inverse of $M$.</b>

The first step is therefore to add suitable multiples of the first row to the other two, to eliminate the first elements of the second and the third row.

For numerical stability, it is generally recommended to swap rows so that the number with the largest magnitude in column 1 is used as the "pivot" (in this first step, the pivot first element the first row.)

For this matrix, the algorithm would work just fine without this row-swapping step. But later when we automate this process, doing this extra step will help with numerical stability: imagine a situation where the 1 in the (1,1) position is instead a 0, or a very small number; using the row with the largest number in the 2nd column will automatically swap this such that we have a larger number in the (1,1) position and thus avoid the crash that would result from division by zero; we also reduce the likelihood of situations where the program has to multiply rows with very awkward numbers like 1/e-13, which tends to lead to numerical inaccuracies.

So, mainly in anticipation of automating this algorithm later, we swap the first row and the 2nd (first with third would have worked as well):

In [22]:
M33_1 = deepcopy(M33)    ## this way we keep all intermediary results. 
I_1 = deepcopy(I)        ## which can be convenient

RowSwap(2, 1, M33_1, I_1)  ## note that this routine modifies the matrices passed to it.
ShowM(M33_1, I_1)

<IPython.core.display.Latex object>

Now we want to eliminate the first elements of the second and third row. 

To eliminate the first element of the second row, we add -1/2 times the first row to the second row:

In [6]:
M33_2 = deepcopy(M33_1)  ## These copies are just there to make debugging easier, but I recommend
I_2 = deepcopy(I_1)      ## you do this, too, when you solve your own problem below. Otherwise,
                         ## confusing things happen when you run the same cell, twice.

RowManip(-1/2, 1, 2, M33_2, I_2)
ShowM(M33_2, I_2)

<IPython.core.display.Latex object>

Let us now eliminate the first element of the third row by adding -1 times the first row to the third.

In [7]:
M33_3 = deepcopy(M33_2)
I_3 = deepcopy(I_2)

RowManip(-1, 1, 3, M33_3, I_3)
ShowM(M33_3, I_3)

<IPython.core.display.Latex object>

Because the first elements of the second and third row are 0, can now add any multiple of the second or third row to any other row without changing the first column, $\begin{pmatrix} 2\\0\\0\end{pmatrix}$.

With this in mind, we can now concentrate on the the second column. Our aim in this next step is to transform the matrix to a shape like this:

$
\begin{pmatrix}
2 & 0 & c \\
0 & d & e \\
0 & 0 & g 
\end{pmatrix}
$

where, now, "d" is our "pivot". For numerical stability, it is generally recommended to use the number with the largest magnitude available as the "pivot", so let's start by swapping the second and third so that "-2" in the becomes our "pivot".

Note that, on this matrix, the algorithm would work just fine without this row-swapping step. But especially later when we automate this process, doing this extra step will help with numerical stability: imagine a situation where the 0.5 in the (2,2) position is instead a 0, or a very small number; using the row with the largest number in the 2nd column will automatically avoid the crash that would result from division by zero, and we also reduce the likelihood of situations where the program has to multiply rows with very awkward numbers like 1/e-13, which tends to lead to numerical inaccuracies.

In [8]:
M33_4 = deepcopy(M33_3)
I_4 = deepcopy(I_3)

RowSwap(2, 3, M33_4, I_4)
ShowM(M33_4, I_4)

<IPython.core.display.Latex object>

Let's now eliminate (set to zero through row manipulations) all elements in the 2nd column, except our "pivot". To achieve this, we add 1/4 of the second row to the third, and 3/2 of the second row to the first:

In [9]:
M33_5 = deepcopy(M33_4)
I_5 = deepcopy(I_4)

RowManip( 1/4, 2, 3, M33_5, I_5)
RowManip( 3/2, 2, 1, M33_5, I_5)
ShowM(M33_5, I_5)

<IPython.core.display.Latex object>

Now we use the last row to eliminate the 3rd elements of the other rows.

In [10]:
M33_6 = deepcopy(M33_5)
I_6 = deepcopy(I_5)

RowManip( 2, 3, 1, M33_6, I_6)
RowManip( 12, 3, 2, M33_6, I_6)
ShowM(M33_6, I_6)

<IPython.core.display.Latex object>

To finish it all off, we multiply the rows so we get ones as the diagonal elements of the matrix on the right.

In [11]:
M33_7 = deepcopy(M33_6)
I_7 = deepcopy(I_6)


MultiplyRow( 1/2, 1, M33_7, I_7)
MultiplyRow(-1/2, 2, M33_7, I_7)
MultiplyRow(  4 , 3, M33_7, I_7)
ShowM(M33_7, I_7)

<IPython.core.display.Latex object>

Now let's see if it worked:

In [12]:
C = MultiplyMatrices(M33, I_7)
ShowM(M33, I_7)
ShowM(C)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Over to you

Can you find the inverse to the matrix shown below?

In [14]:
N33= [[6, 2, 1]
     ,[1, -6, -1]
     ,[-2, 1, 0]]

ShowM(N33)

<IPython.core.display.Latex object>

Let's start by creating our own unit matrix

In [16]:
myI =[[1, 0, 0]
     ,[0, 1, 0]
     ,[0, 0, 1]]

ShowM(N33, myI)

<IPython.core.display.Latex object>

The next step is your first row manipulation to solve this problem. The cell below is incomplet.

In [21]:
N33_1 = deepcopy(N33)
myI_1 = deepcopy(myI)

RowManip(   , 1, 2, N33_1, myI_1)  # enter the missing number (hint: it's -1/6)

ShowM(N33_1, myI_1)

SyntaxError: invalid syntax (3618107272.py, line 4)

That got us the a zero in the 2nd row, first column. Now let's get a zero in the 3rd row, first column:

In [20]:
N33_2 = deepcopy(N33_1)
myI_2 = deepcopy(myI_1)

RowManip(  , 1, 3, N33_2, myI_2)  # enter the missing number (now it's really over to you)

ShowM(N33_2, myI_2)

SyntaxError: invalid syntax (4167257793.py, line 4)